In [19]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

In [24]:
np.random.seed(123)

In [25]:
#----------------------------------------------------------------------
##  BLOCK OF FUNCTIONS USED IN THE MAIN CODE
#----------------------------------------------------------------------
def initialstate(N):   
    ''' generates a random spin configuration for initial condition'''
    state = 2*np.random.randint(2, size=(N,N))-1
    return state


def mcmove(config, T):
    '''Monte Carlo move using Metropolis algorithm '''
    deltaE=0
    deltaM=0
    for i in range(N):
        for j in range(N):
                a = np.random.randint(0, N)
                b = np.random.randint(0, N)
                s =  config[a, b]
                nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]
                cost = 2*s*nb
                if cost < 0:
                    s *= -1
                elif rand() < np.exp(-cost/T):
                    s *= -1
                else:
                    cost=0
                deltaE+=cost
                deltaM -= config[a,b]-s
                config[a, b] = s
    return [config,deltaE,deltaM]


def calcEnergy(config):
    '''Energy of a given configuration'''
    energy = 0
    rows,columns=config.shape
    for i in range(rows):
        for j in range(columns):
            energy-= config[i,j]*config[(i+1)%N,j] + config[i,j]*config[i,(j+1)%N]
    return energy


def calcMag(config):
    '''Magnetization of a given configuration'''
    mag = np.sum(config)
    return mag

In [26]:
## change these parameters for a smaller (faster) simulation 
nt      = 1         #  number of temperature points
N       = 8         #  size of the lattice, N x N
eqSteps = 10000       #  number of MC sweeps for equilibration
mcSteps = 50000       #  number of MC sweeps for calculation

T       = 2.269186; 
#E,M,C,X = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
#n1, n2  = 1.0/(mcSteps*N*N), 1.0/(mcSteps*mcSteps*N*N) 
# divide by number of samples, and by system size to get intensive values

In [27]:
#----------------------------------------------------------------------
#  MAIN PART OF THE CODE
#----------------------------------------------------------------------
outputFile = open('2dising_%s.txt'%N, 'w')
outputFile.write('step\tMagnetization\tEnergy\n')

config = initialstate(N)
Ene = calcEnergy(config)     # calculate the energy
Mag = calcMag(config)        # calculate the magnetisation


for i in range(eqSteps):         # equilibrate
    config, deltaE, deltaM = mcmove(config, T)           # Monte Carlo moves
    Ene+=deltaE
    Mag+=deltaM
    #print('%s : %s'%(Mag,calcMag(config)))

for i in range(mcSteps):
    config, deltaE, deltaM = mcmove(config, T)           
    Ene+=deltaE
    Mag+=deltaM
    outputFile.write('%s\t%0.6f\t%s\n'%(i,abs(Mag)/(N*N),Ene))

outputFile.close()

In [ ]:
f = plt.figure(figsize=(18, 10)); # plot the calculated values    

sp =  f.add_subplot(2, 2, 1 );
plt.scatter(T, E, s=50, marker='o', color='IndianRed')
plt.xlabel("Temperature (T)", fontsize=20);
plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');

sp =  f.add_subplot(2, 2, 2 );
plt.scatter(T, abs(M), s=50, marker='o', color='RoyalBlue')
plt.xlabel("Temperature (T)", fontsize=20); 
plt.ylabel("Magnetization ", fontsize=20);   plt.axis('tight');

sp =  f.add_subplot(2, 2, 3 );
plt.scatter(T, C, s=50, marker='o', color='IndianRed')
plt.xlabel("Temperature (T)", fontsize=20);  
plt.ylabel("Specific Heat ", fontsize=20);   plt.axis('tight');   

sp =  f.add_subplot(2, 2, 4 );
plt.scatter(T, X, s=50, marker='o', color='RoyalBlue')
plt.xlabel("Temperature (T)", fontsize=20); 
plt.ylabel("Susceptibility", fontsize=20);   plt.axis('tight');